In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import scipy.sparse

# Descargar recursos necesarios para NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Cargar el dataset unificado
dataset_unificado = pd.read_csv("dataset_unificado.csv")

In [8]:


# Configuración de stopwords y lematización
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

def procesar_texto_complejo(texto):
    """
    Preprocesa el texto eliminando URLs, caracteres especiales, stopwords y aplicando lematización.

    Args:
        texto (str): Texto a preprocesar.

    Returns:
        str: Texto preprocesado.
    """
    texto = re.sub(r'http\S+', '', texto)  # Eliminar URLs
    texto = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]', '', texto)  # Eliminar caracteres especiales
    texto = texto.lower()  # Convertir a minúsculas
    palabras = texto.split()
    palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras if palabra not in stop_words]  # Lematizar y eliminar stopwords
    return ' '.join(palabras)

# Aplicar preprocesamiento al contenido y título
dataset_unificado["Contenido"] = dataset_unificado["Contenido"].apply(procesar_texto_complejo)
dataset_unificado["Título"] = dataset_unificado["Título"].apply(procesar_texto_complejo)

# Guardar dataset preprocesado
dataset_unificado.to_csv("dataset_unificado_preprocesado.csv", index=False)

# Representar el texto usando TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(dataset_unificado["Contenido"] + " " + dataset_unificado["Título"])

# Guardar el TF-IDF para futuros usos
scipy.sparse.save_npz("X_tfidf.npz", X_tfidf)

# División de los datos para entrenamiento, validación y prueba
y = dataset_unificado["etiqueta"]
X_train, X_temp, y_train, y_temp = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\iagob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\iagob\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iagob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

# Guardar datos procesados
scipy.sparse.save_npz("X_train_tfidf.npz", X_train)
scipy.sparse.save_npz("X_val_tfidf.npz", X_val)
scipy.sparse.save_npz("X_test_tfidf.npz", X_test)
y_train.to_csv("y_train_final.csv", index=False)
y_val.to_csv("y_val_final.csv", index=False)
y_test.to_csv("y_test_final.csv", index=False)

print("Preprocesamiento avanzado completado y datos guardados.")